In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd

# SQLite 데이터베이스 파일명
DB_NAME = 'mainichi_editorials.db'

# 데이터베이스 연결 및 커서 생성
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# 테이블에 content 컬럼이 있는지 확인하고 없으면 추가
cursor.execute("PRAGMA table_info(editorials)")
columns = [column[1] for column in cursor.fetchall()]
if 'content' not in columns:
    cursor.execute("ALTER TABLE editorials ADD COLUMN content TEXT")
    conn.commit()

# 데이터프레임으로 테이블 데이터 불러오기
df = pd.read_sql('SELECT * FROM editorials', conn)
df = df[df["title"] != "No Title"]
# content가 비어 있는 경우 URL을 통해 데이터 수집
def get_article_content(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        
        article_body = soup.select("#main > div > article")
    
        if article_body:
            paragraphs = article_body[0].find_all('p')
            content = '\n'.join([p.get_text(strip=True) for p in paragraphs])
        else:
            content = "본문을 찾을 수 없습니다."
        
        return content
    
    except requests.RequestException as e:
        return f"요청 중 오류 발생: {e}"
    except Exception as e:
        return f"예상치 못한 오류 발생: {e}"

# content가 비어 있는 행에 대해 데이터를 수집하고 업데이트
for index, row in df.iterrows():
    if pd.isna(row['content']) or (row['content'] == '') or (row['content'].strip() == "Content not found"):
        content = get_article_content(row['link'])
        cursor.execute("UPDATE editorials SET content = ? WHERE id = ?", (content, row['id']))
        conn.commit()
        time.sleep(1)  # 너무 많은 요청을 방지하기 위해 잠시 대기

# 데이터베이스 연결 종료
conn.close()

In [ ]:
# SQLite 데이터베이스 파일명
DB_NAME = 'mainichi_editorials.db'

# 데이터베이스 연결 및 커서 생성
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()


# 데이터프레임으로 테이블 데이터 불러오기
df = pd.read_sql('SELECT * FROM editorials', conn)
# 데이터베이스 연결 종료
conn.close()
df

In [ ]:
df[df.duplicated()]

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.iloc[1]['content']